# Purpose

The purpose of this notebook is to run an example experiment using the functions from `fsnet-tools.py` to illustrate it's intended use.

# Data

A wide toy dataset will be generated using `make_regression` from `sklearn`.  I'll then split off a test dataset, standardize, and split off a validation set.

In [1]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

# generate example regression dataset
X, y = make_regression(n_samples=500, n_features=4000, n_informative=25)

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2)


In [2]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_full_scaled = scaler.fit_transform(X_train_full)
X_test = scaler.transform(X_test)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full_scaled, y_train_full)

print("train shape - X:", X_train.shape, ", y:", y_train.shape)
print("valid shape - X:", X_valid.shape, ", y:", y_valid.shape)
print("test shape - X:", X_test.shape, ", y:", y_test.shape)


train shape - X: (300, 4000) , y: (300,)
valid shape - X: (100, 4000) , y: (100,)
test shape - X: (100, 4000) , y: (100,)


# Build Model

Now I'll build the model using the newly created `fsnet-tools`.  `u_train` and `alpha` should be generated first, using the respective functions.  They will only need to be regenerated if `X_train` changes.  This is important to note, say, inside a cross-fold validation experiment.  

It's also important that hyperparameters match between the various function calls.  Here, I'm defining a hyperparameter dictionary, as this is consistent with my typical model experiment workflow.  This method also lends itself well to hyperparameter tuning via optimization algorithms.

In [3]:
from fsnet_tools import get_utrain
from fsnet_tools import get_alpha
from fsnet_tools import build_model

hps = {
    "bins": 10,
    "min_temp": 0.01,
    "start_temp": 10.0,
    "num_epochs": 500,
    "nfeat": 50,
    "h_size": 16,
}

u_train = get_utrain(X_train, bins=hps["bins"])
alpha = get_alpha(
    X_train,
    min_temp=hps["min_temp"],
    start_temp=hps["start_temp"],
    num_epochs=hps["num_epochs"],
)
model = build_model(
    num_inputs=X_train.shape[1],
    nfeat=hps["nfeat"],
    u_train=u_train,
    alpha=alpha,
    h_size=hps["h_size"],
    bins=hps["bins"],
    start_temp=hps["start_temp"],
    min_temp=hps["min_temp"],
    num_epochs=hps["num_epochs"],
)


Using TensorFlow backend.














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



# Fit

Now I'll fit the data with the model defined above.

In [4]:
history = model.fit(
    X_train,
    {"recon": X_train, "reg_output": y_train},
    validation_data=(X_valid, {"recon": X_valid, "reg_output": y_valid}),
    epochs=hps["num_epochs"],
    verbose=1,
)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 300 samples, validate on 100 samples
Epoch 1/500
300/300 [==============================] - 1s 5ms/step - loss: 92983.3499 - recon_loss: 1.0014 - reg_output_loss: 92883.2120 - recon_mean_squared_error: 1.0014 - reg_output_mean_squared_error: 92883.2120 - val_loss: 66879.4795 - val_recon_loss: 1.6952 - val_reg_output_loss: 66709.9612 - val_recon_mean_squared_error: 1.6952 - val_reg_output_mean_squared_error: 66709.9612
Epoch 2/500
300/300 [==============================] - 0s 223us/step - loss: 92980.8934 - recon_loss: 1.0008 - reg_output_loss: 92880.8186 - recon_mean_squared_error: 1.0008 - reg_output_mean_squared_error: 92880.8186 - val_loss: 66846.3547 - val_recon_loss: 1.3435 - val_reg_output_loss: 66712.0086 - val_recon_mean_squared_error: 1.3435 - val_reg_output_mean_squared_error: 66712.0086
Epoch 3/500
300/300 [==============================] - 0s 220us/step - loss: 92981.5574

# Test

And now I'll evaluate the model on train, valid, and test sets.

In [10]:
print(
    "train MSE:",
    model.evaluate(X_train, {"recon": X_train, "reg_output": y_train}, verbose=0)[4],
)
print(
    "valid MSE:",
    model.evaluate(X_valid, {"recon": X_valid, "reg_output": y_valid}, verbose=0)[4],
)
print(
    "test MSE: ",
    model.evaluate(X_test, {"recon": X_test, "reg_output": y_test}, verbose=0)[4],
)


train MSE: 108261.52645833333
valid MSE: 92234.3953125
test MSE:  110075.475625
